In [12]:
import mediapipe as mp
import cv2
import numpy as np
import time
import random

mphands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


In [13]:


def begin(video):
    while True:
        ret, img = video.read()
        if not ret:
            continue

        img = cv2.flip(img, 1)
        frame = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
        height, width = frame.shape[:2]

        # Vibrant background (dark blue)
        blank = np.zeros((height, width, 3), dtype=np.uint8)
        blank[:] = (20, 30, 60)

        # Main title
        title = "ODD or EVEN"
        subtitle = "Press 'Q' to Begin the Game"
        note = "Make sure your camera is ON and hands are visible"

        # Text styles
        font = cv2.FONT_HERSHEY_SIMPLEX

        # Title
        title_size = cv2.getTextSize(title, font, 2.2, 5)[0]
        title_x = (width - title_size[0]) // 2
        cv2.putText(blank, title, (title_x, height // 3), font, 2.2, (0, 255, 200), 5)

        # Subtitle
        sub_size = cv2.getTextSize(subtitle, font, 1.2, 3)[0]
        sub_x = (width - sub_size[0]) // 2
        cv2.putText(blank, subtitle, (sub_x, height // 2), font, 1.2, (255, 255, 255), 3)

        # Note (optional helper text)
        note_size = cv2.getTextSize(note, font, 0.8, 2)[0]
        note_x = (width - note_size[0]) // 2
        cv2.putText(blank, note, (note_x, int(height * 0.65)), font, 0.8, (150, 200, 255), 2)

        # Show screen
        cv2.imshow("Welcome Screen", blank)

        # Wait for Q key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyWindow("Welcome Screen")
            return True



# Thumbs up detection screen with instruction + live video feed
import cv2
import time
import mediapipe as mp

mphands = mp.solutions.hands

def intro(video, lines):
    with mphands.Hands(max_num_hands=1, min_detection_confidence=0.7) as hands:
        hold_start = None
        hold_required = 2  # seconds

        while True:
            ret, img = video.read()
            if not ret:
                continue

            img = cv2.flip(img, 1)
            frame = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
            height, width = frame.shape[:2]
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            # Background
            bg = frame.copy()
            bg[:] = (15, 25, 40)

            # Instruction text
            y_start = 80
            spacing = 60
            for i, line in enumerate(lines):
                text_size = cv2.getTextSize(line, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)[0]
                x = (width - text_size[0]) // 2
                y = y_start + i * spacing
                cv2.putText(bg, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

            # Detect thumbs-up gesture
            detected = False
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    lm = [(lm.x, lm.y) for lm in hand_landmarks.landmark]

                    thumb_up = lm[4][1] < lm[3][1]
                    fingers_down = all(lm[i][1] > lm[i - 2][1] for i in [8, 12, 16, 20])

                    if thumb_up and fingers_down:
                        detected = True
                        if hold_start is None:
                            hold_start = time.time()
                        elapsed = time.time() - hold_start

                        # Show hold time countdown
                        countdown = f"Holding: {elapsed:.1f}s / {hold_required}s"
                        countdown_size = cv2.getTextSize(countdown, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
                        cx = (width - countdown_size[0]) // 2
                        cy = y_start + len(lines) * spacing + 40
                        cv2.putText(bg, countdown, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

                        if elapsed >= hold_required:
                            success_msg = " Thumbs Up Confirmed!"
                            size = cv2.getTextSize(success_msg, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)[0]
                            x = (width - size[0]) // 2
                            y = cy + 60
                            cv2.putText(bg, success_msg, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
                            cv2.imshow("Intro", bg)
                            cv2.waitKey(1200)
                            cv2.destroyWindow("Intro")
                            return True
                    else:
                        hold_start = None

            if not detected:
                hold_start = None

            cv2.imshow("Intro", bg)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyWindow("Intro")
                return False


In [14]:
def show_text_with_timer(video, display_text, seconds=5):
    import textwrap

    # Capture a frame to get size
    ret, img = video.read()
    if not ret:
        print("❌ Camera read failed.")
        return False

    frame = cv2.resize(img, None, fx=1.5, fy=1.5)
    height, width = frame.shape[:2]
    start_time = time.time()

    # Ensure text is a list
    if isinstance(display_text, str):
        display_text = [display_text]

    # Word wrapping for long lines
    wrapped_lines = []
    for line in display_text:
        wrapped_lines.extend(textwrap.wrap(line, width=30))

    while True:
        screen = np.zeros((height, width, 3), dtype=np.uint8)
        screen[:] = (20, 20, 20)  # dark background

        elapsed = time.time() - start_time
        remaining = int(seconds - elapsed)

        # Draw each text line centered
        y_start = height // 3
        for i, line in enumerate(wrapped_lines):
            text_size = cv2.getTextSize(line, cv2.FONT_HERSHEY_SIMPLEX, 1.4, 3)[0]
            x = (width - text_size[0]) // 2
            y = y_start + i * 60
            cv2.putText(screen, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255, 255, 255), 3)

        # Countdown timer
        if remaining >= 0:
            timer_text = f"Starting in: {remaining}"
            timer_size = cv2.getTextSize(timer_text, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2)[0]
            x = (width - timer_size[0]) // 2
            y = y_start + len(wrapped_lines) * 60 + 40
            cv2.putText(screen, timer_text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2)
        else:
            break

        cv2.imshow("Blank Screen", screen)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            cv2.destroyWindow("Blank Screen")
            return False

    cv2.destroyWindow("Blank Screen")
    return True


In [15]:
import cv2
import mediapipe as mp
import time

mphands = mp.solutions.hands

def thumbs_up(video):
    tip_id = [4, 8, 12, 16, 20]
    hold_start = None
    hold_duration = 2  # seconds

    with mphands.Hands(max_num_hands=1, min_detection_confidence=0.7) as hands:
        while True:
            success, img = video.read()
            if not success:
                continue

            frame = cv2.flip(img, 1)
            frame = cv2.resize(frame, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
            h, w = frame.shape[:2]
            frame[:] = (25, 30, 35)  # Dark background
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            result = hands.process(img_rgb)
            lst = []
            thumbs_detected = False

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    for id, lm in enumerate(hand_landmarks.landmark):
                        lst.append([id, lm.x, lm.y])

                    if len(lst) == 21:
                        thumb_up = lst[4][1] < lst[3][1]
                        fingers_down = all(lst[tip_id[i]][2] > lst[tip_id[i] - 2][2] for i in range(1, 5))

                        if thumb_up and fingers_down:
                            thumbs_detected = True
                            if hold_start is None:
                                hold_start = time.time()
                            elapsed = time.time() - hold_start

                            # Countdown text
                            count_text = f"Holding Thumbs Up: {elapsed:.1f}s / {hold_duration}s"
                            count_size = cv2.getTextSize(count_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
                            cx = (w - count_size[0]) // 2
                            cv2.putText(frame, count_text, (cx, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 200), 2)

                            if elapsed >= hold_duration:
                                confirm_msg = "✅ Thumbs Up Confirmed!"
                                msg_size = cv2.getTextSize(confirm_msg, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)[0]
                                x = (w - msg_size[0]) // 2
                                y = h // 2
                                cv2.putText(frame, confirm_msg, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 4)
                                cv2.imshow("Live Camera Feed", frame)
                                cv2.waitKey(1000)
                                cv2.destroyWindow("Live Camera Feed")
                                return True
                        else:
                            hold_start = None

            if not thumbs_detected:
                hold_start = None

            # Instruction text (always shown)
            instruction = "Show a Thumbs Up to Continue..."
            ins_size = cv2.getTextSize(instruction, cv2.FONT_HERSHEY_SIMPLEX, 1.1, 2)[0]
            x = (w - ins_size[0]) // 2
            cv2.putText(frame, instruction, (x, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (255, 255, 255), 2)

            cv2.imshow("Live Camera Feed", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyWindow("Live Camera Feed")
                return False


In [16]:
import cv2
import mediapipe as mp
import time

def choose_odd_even(video, lines):
    mphands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mphands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    tip_id = [4, 8, 12, 16, 20]
    detection_start_time = None
    required_time = 5  # seconds

    while True:
        ret, frame = video.read()
        if not ret:
            continue

        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, None, fx=1.5, fy=1.5)
        height, width = frame.shape[:2]

        # Draw translucent overlay box
        overlay = frame.copy()
        cv2.rectangle(overlay, (40, 40), (width - 40, 400), (0, 0, 0), -1)
        alpha = 0.45
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

        # Draw instruction text centered
        spacing = 50
        for i, line in enumerate(lines):
            text_size = cv2.getTextSize(line, cv2.FONT_HERSHEY_SIMPLEX, 1.1, 2)[0]
            x = (width - text_size[0]) // 2
            y = 80 + i * spacing
            cv2.putText(frame, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (255, 255, 255), 2)

        # Static holding message
        hold_msg = "Hold gesture for 5 seconds"
        hold_size = cv2.getTextSize(hold_msg, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
        hold_x = (width - hold_size[0]) // 2
        cv2.putText(frame, hold_msg, (hold_x, 350), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

        # Process hand input
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        current_count = 0
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                landmarks = [[id, lm.x, lm.y] for id, lm in enumerate(hand_landmarks.landmark)]

                if len(landmarks) == 21:
                    fingers = []
                    for i in range(1, 5):  # index to pinky
                        fingers.append(1 if landmarks[tip_id[i]][2] < landmarks[tip_id[i] - 2][2] else 0)

                    count = sum(fingers)
                    current_count = count if count in [2, 3] else 0

                    if current_count > 0:
                        label = "EVEN" if current_count == 2 else "ODD"

                        # Detected count text
                        detect_text = f"Detected: {current_count}"
                        detect_size = cv2.getTextSize(detect_text, cv2.FONT_HERSHEY_COMPLEX, 1.8, 3)[0]
                        dx = (width - detect_size[0]) // 2
                        cv2.putText(frame, detect_text, (dx, 460), cv2.FONT_HERSHEY_COMPLEX, 1.8, (255, 0, 0), 3)

                        # Label text (ODD/EVEN)
                        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_COMPLEX, 2.5, 4)[0]
                        lx = (width - label_size[0]) // 2
                        cv2.putText(frame, label, (lx, 260), cv2.FONT_HERSHEY_COMPLEX, 2.5, (0, 255, 0), 4)

                        if detection_start_time is None:
                            detection_start_time = time.time()
                        else:
                            elapsed = time.time() - detection_start_time
                            hold_text = f"Holding: {int(elapsed)}s"
                            hold_size = cv2.getTextSize(hold_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
                            hx = (width - hold_size[0]) // 2
                            cv2.putText(frame, hold_text, (hx, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

                            if elapsed >= required_time:
                                hands.close()
                                cv2.destroyWindow("odd_or_even")
                                return label
                    else:
                        detection_start_time = None
        else:
            detection_start_time = None

        cv2.imshow("odd_or_even", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    hands.close()
    cv2.destroyWindow("odd_or_even")
    return None


In [17]:


def Toss(video):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

    required_hold_time = 3  # Hold gesture for 3 seconds
    show_opponent_delay = 2  # Delay before showing opponent number

    tip_id = [4, 8, 12, 16, 20]
    current_count = None
    detection_start_time = None
    random_num = None
    show_opponent_time = None

    while True:
        ret, img = video.read()
        if not ret:
            continue

        frame = cv2.flip(img, 1)
        frame = cv2.resize(frame, None, fx=1.5, fy=1.5)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        result = hands.process(img_rgb)
        lst = []
        finger_count = 0

        if result.multi_hand_landmarks:
            for hand in result.multi_hand_landmarks:
                for id, lm in enumerate(hand.landmark):
                    lst.append([id, lm.x, lm.y])

                if len(lst) == 21:
                    fingers = []

                    # Check if it's thumbs-up gesture (for 6)
                    thumbs_up = lst[4][2] < lst[3][2]
                    others_down = all(lst[tip_id[i]][2] > lst[tip_id[i] - 2][2] for i in range(1, 5))
                    if thumbs_up and others_down:
                        fingers = [1] * 6  # Count thumbs-up as 6
                    else:
                        # Thumb logic: tip right of joint (after mirror) and tip higher
                        thumb_extended = lst[4][0] > lst[3][0] and lst[4][1] < lst[3][1]
                        fingers.append(1 if thumb_extended else 0)

                        # Index to pinky
                        for i in range(1, 5):
                            fingers.append(1 if lst[tip_id[i]][2] < lst[tip_id[i] - 2][2] else 0)

                    finger_count = sum(fingers)

                    # Holding detection
                    if finger_count == current_count:
                        if detection_start_time is None:
                            detection_start_time = time.time()
                        elapsed = time.time() - detection_start_time
                        cv2.putText(frame, f"Holding: {int(elapsed)}s", (50, 450),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                        if elapsed >= required_hold_time:
                            if random_num is None:
                                random_num = random.randint(1, 6)
                                show_opponent_time = time.time()
                    else:
                        current_count = finger_count
                        detection_start_time = time.time()

        # After holding, show result
        if random_num is not None:
            delay_elapsed = time.time() - show_opponent_time
            if delay_elapsed >= show_opponent_delay:
                cv2.putText(frame, f"Opponent: {random_num}", (30, 330),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3)
                cv2.putText(frame, f"You: {finger_count}", (30, 400),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 3)
                print("You:", finger_count, "System:", random_num)
                cv2.imshow("Hands", frame)
                cv2.waitKey(2000)
                hands.close()
                cv2.destroyWindow("Hands")
                return finger_count + random_num

        else:
            cv2.putText(frame, f"Hold gesture for {required_hold_time} seconds", (30, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

        cv2.imshow("Hands", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    hands.close()
    cv2.destroyWindow("Hands")
    return None


In [18]:
import cv2
import time
import mediapipe as mp

def batting_or_bowl(video):
    mphands = mp.solutions.hands
    hands = mphands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils

    batting_box = ((150, 200), (350, 350))
    bowling_box = ((500, 200), (700, 350))

    batting_start = None
    bowling_start = None
    required_time = 3
    selection = None

    def draw_fancy_box(frame, top_left, bottom_right, color, text, glow=False):
        if glow:
            cv2.rectangle(frame, top_left, bottom_right, (255, 255, 255), 8)
        cv2.rectangle(frame, top_left, bottom_right, color, 4)

        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 1.1
        thick = 3
        size, _ = cv2.getTextSize(text, font, scale, thick)
        x = top_left[0] + ((bottom_right[0] - top_left[0] - size[0]) // 2)
        y = bottom_right[1] + 50
        cv2.putText(frame, text, (x, y), font, scale, color, thick)

    while True:
        ret, frame = video.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, None, fx=1.5, fy=1.5)
        h, w, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)

        # Header instruction
        cv2.putText(frame, "👉 Move your index finger into a box to choose",
                    (80, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 255, 200), 2)

        holding_text = ""
        glow_batting, glow_bowling = False, False

        if result.multi_hand_landmarks:
            for lm in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, lm, mphands.HAND_CONNECTIONS)
                x = int(lm.landmark[8].x * w)
                y = int(lm.landmark[8].y * h)
                cv2.circle(frame, (x, y), 12, (255, 255, 0), -1)

                if batting_box[0][0] <= x <= batting_box[1][0] and batting_box[0][1] <= y <= batting_box[1][1]:
                    if batting_start is None:
                        batting_start = time.time()
                    elapsed = time.time() - batting_start
                    holding_text = f"Holding: {int(elapsed)}s"
                    glow_batting = True
                    if elapsed >= required_time:
                        selection = "Batting"
                        break
                else:
                    batting_start = None

                if bowling_box[0][0] <= x <= bowling_box[1][0] and bowling_box[0][1] <= y <= bowling_box[1][1]:
                    if bowling_start is None:
                        bowling_start = time.time()
                    elapsed = time.time() - bowling_start
                    holding_text = f"Holding: {int(elapsed)}s"
                    glow_bowling = True
                    if elapsed >= required_time:
                        selection = "Bowling"
                        break
                else:
                    bowling_start = None

        draw_fancy_box(frame, batting_box[0], batting_box[1], (0, 200, 0), "Batting", glow_batting)
        draw_fancy_box(frame, bowling_box[0], bowling_box[1], (0, 0, 255), "Bowling", glow_bowling)

        if holding_text:
            cv2.putText(frame, holding_text, (w // 2 - 100, h - 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)

        cv2.imshow("Batting or Bowling?", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if selection:
            print("✅ Selected:", selection)
            time.sleep(1)
            break

    hands.close()
    cv2.destroyWindow("Batting or Bowling?")
    return selection


In [19]:
def draw_text_box(frame, text, position, font_scale=0.9, text_color=(255, 255, 255),
                  bg_color=(50, 50, 50), thickness=2, padding=10, alpha=0.6):
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    x, y = position
    box_coords = ((x - padding, y - text_size[1] - padding),
                  (x + text_size[0] + padding, y + padding))

    overlay = frame.copy()
    cv2.rectangle(overlay, box_coords[0], box_coords[1], bg_color, -1)
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    cv2.putText(frame, text, (x, y), font, font_scale, text_color, thickness)




def play(video,who,target):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    tip_ids = [4, 8, 12, 16, 20]

    desired_width, desired_height = 960, 720

    values = [0]
    total = 0
    storing = False
    black_screen_start = None
    system_number = None
    confirmed_number = None
    timer_start = None
    result_message = None

    while True:
        ret, frame = video.read()
        if not ret:
            continue

        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (desired_width, desired_height))
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        # Game info
        role_text = f"Batting: {'User' if who == 'user' else 'System'} | Bowling: {'System' if who == 'user' else 'User'}"
        frame_height, frame_width = frame.shape[:2]

        draw_text_box(frame, role_text, (20, 60), font_scale=0.9, text_color=(255, 255, 0))
        draw_text_box(frame, f"Total: {total}", (20, 110), font_scale=0.9, text_color=(0, 255, 255))
        draw_text_box(frame, f"Balls: {len(values)}", (20, 160), font_scale=0.9, text_color=(255, 0, 255))


        # Allocate Target to top-right corner
        if target is not None:
            target_text = f" Target: {target}"
            text_size, _ = cv2.getTextSize(target_text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)
            draw_text_box(frame, target_text, (frame_width - text_size[0] - 20, 60), text_color=(0, 200, 255))



        # Show black screen if storing or result
        if storing:
            black_elapsed = time.time() - black_screen_start
            black_frame = np.zeros((desired_height, desired_width, 3), dtype=np.uint8)

            if result_message:
                cv2.putText(black_frame, result_message, (250, 330), cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 255, 0), 4)
                cv2.putText(black_frame, f"Final Total: {total}", (300, 420), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
                cv2.imshow("Gesture Capture", black_frame)
                cv2.waitKey(3000)
                break

            if black_elapsed < 2:
                cv2.putText(black_frame, "Next Ball", (320, 330), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
                cv2.imshow("Gesture Capture", black_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                continue
            else:
                storing = False
                confirmed_number = None
                timer_start = None

        count = 0
        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:
                lm_list = [(lm.x, lm.y) for lm in handLms.landmark]

                if len(lm_list) == 21:
                    # Detect 6 for thumbs up
                    thumb_up = lm_list[4][1] < lm_list[3][1]
                    others_down = all(lm_list[tip_ids[i]][1] > lm_list[tip_ids[i] - 2][1] for i in range(1, 5))

                    if thumb_up and others_down:
                        count = 6
                    else:
                        fingers = []
                        fingers.append(1 if lm_list[4][0] < lm_list[3][0] else 0)
                        for i in range(1, 5):
                            fingers.append(1 if lm_list[tip_ids[i]][1] < lm_list[tip_ids[i] - 2][1] else 0)
                        count = sum(fingers)

                    draw_text_box(frame, f"Detected: {count}", (50, desired_height - 60), font_scale=1.2, text_color=(255, 255, 0))

                    if confirmed_number != count and count != 0:
                        confirmed_number = count
                        timer_start = time.time()
                    elif confirmed_number == count and timer_start:
                        elapsed = time.time() - timer_start
                        cv2.putText(frame, f"Hold for: {int(3 - elapsed)}s", (50, 550), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                        if elapsed >= 3:
                            system_number = random.randint(1, 6)
                            print(f"🧍 You: {count},  System: {system_number}")

                            if count == system_number:
                                result_message = "OUT !!!!"
                                storing = True
                                black_screen_start = time.time()
                                continue

                            values.append(count)
                            total += count if who == "user" else system_number

                            # Check if won
                            if target is not None and total > target:
                                result_message = " WON!"
                                storing = True
                                black_screen_start = time.time()
                                continue

                            storing = True
                            black_screen_start = time.time()
                    else:
                        timer_start = None
                        confirmed_number = None
        else:
            timer_start = None
            confirmed_number = None

        if system_number:
            draw_text_box(frame, f"System: {system_number}", (desired_width - 280, desired_height - 60), font_scale=1.2, text_color=(0, 255, 0))

        cv2.imshow("Gesture Capture", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # cap.release()
    hands.close()
    cv2.destroyAllWindows()
    print("✅ Final Total:", total)
    return total


In [20]:
def show_end_note(video, message, seconds=4):
    import textwrap
    import time

    # Read a frame to get dimensions
    ret, img = video.read()
    if not ret:
        print("❌ Failed to read from camera.")
        return False

    frame = cv2.resize(img, None, fx=1.5, fy=1.5)
    height, width = frame.shape[:2]
    end_time = time.time() + seconds

    # Ensure message is a list
    if isinstance(message, str):
        message = [message]

    # Word wrap long messages
    wrapped_lines = []
    for line in message:
        wrapped_lines.extend(textwrap.wrap(line, width=30))

    while time.time() < end_time:
        screen = np.zeros((height, width, 3), dtype=np.uint8)
        screen[:] = (0, 0, 40)  # Dark navy background

        y_start = height // 3
        for i, line in enumerate(wrapped_lines):
            text_size = cv2.getTextSize(line, cv2.FONT_HERSHEY_SIMPLEX, 1.4, 3)[0]
            x = (width - text_size[0]) // 2
            y = y_start + i * 60
            cv2.putText(screen, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 3)

        cv2.imshow("End Note", screen)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cv2.destroyWindow("End Note")
    return True


In [21]:
# Main control function
def full():
    video = cv2.VideoCapture(0)  # Use 0 unless you know index 2 is correct
    if not video.isOpened():
        print("❌ Cannot open camera")
        return

    begin_value = begin(video)
    if begin_value:
        choose = intro(video, [
            "ODD or EVEN",
            "Choose on next Page",
            "(ODD = 3 fingers, EVEN = 2 fingers)",
            "Show thumbs up to continue"
        ])
        
        if choose:
            odd_even = choose_odd_even(video, ["(ODD = 3 fingers, EVEN = 2 fingers)"]) 
            if odd_even:
                print("✅ Result:", odd_even)
                messages = [
                f"You chose {odd_even}",
                "TOSS",
                "Show the number 1 to 6 ...",
                ]
                
                O_E = 0 if odd_even=="EVEN" else 1
                show_text_with_timer(video,messages)
                toss = Toss(video)
                print(toss)
                if (toss%2==O_E==0) or (toss%2==1==O_E):
                    show_text_with_timer(video,["Selection Upto You","Select Batting Or Bowling On Next..."])
                    bat_or_bowl = batting_or_bowl(video)
                    # print(bat_or_bowl)
                    show_text_with_timer(video,[f"You Chose chose {bat_or_bowl}"])
                    show_text_with_timer(video,["First Innings"])
                    if bat_or_bowl == "Batting":
                        first_innings = play(video,"user",None)
                        show_text_with_timer(video,[f"You Scored {first_innings}","Now Second Innings"])
                        second_innings = play(video,"system",first_innings)
                        if first_innings > second_innings:
                            show_text_with_timer(video,["You Won  !!!!","Where's the Treat ??"])
                            show_end_note(video, [ "Thanks for Playing!"])

                        elif first_innings < second_innings:
                            show_text_with_timer("System Won ","Don't worry , Better Luck Next time") 
                            show_end_note(video, [ "Thanks for Playing!"])  
                        else:
                            show_text_with_timer(video,["That's a Tie","What do we do now ?","Play Again"]) 
                            play(video,"user",None)   
                    else:
                        first_innings = play(video,"system",None)
                        show_text_with_timer(video,[f"System Scored {first_innings}",f"You Need to Score More than {first_innings} ","Now Second Innings"])
                        second_innings = play(video,"user",first_innings)
                        if first_innings > second_innings:
                            show_text_with_timer(video,["System Won ","Don't worry , Better Luck Next time"])
                            show_end_note(video, [ "Thanks for Playing!"]) 
                        elif first_innings < second_innings :
                            show_text_with_timer(["You Won  !!!!","Where's the Treat ??"])
                            show_end_note(video, [ "Thanks for Playing!"])
                        else:
                            show_text_with_timer(video,["That's a Tie","What do we do now ?","Play Again"]) 
                            play(video,"System",None)              
                else:
                    bat_or_bowl = random.choice(["Batting","Bowling"])
                    show_text_with_timer(video,['Selection upto Computer',f"Computer Chose  {bat_or_bowl}"])
                    show_text_with_timer(video,["First Innings"])
                    if bat_or_bowl == "Batting":
                        
                        first_innings = play(video,"system",None)
                        show_text_with_timer(video,[f"System Scored {first_innings}",f"You Need to Score More than {first_innings} ","Now Second Innings"])
                        second_innings = play(video,"user",first_innings)
                        if first_innings > second_innings:
                            show_text_with_timer(video,["You Loose "])
                            show_end_note(video, [ "Thanks for Playing!"]) 
                        elif first_innings < second_innings:
                            show_text_with_timer(video,["You Won  !!!!","Treat ???"]) 
                            show_end_note(video, [ "Thanks for Playing!"]) 
                        else:
                            show_text_with_timer(video,["That's a Tie","What do we do now ?","Play Again"]) 
                            play(video,"system",None)    
                    else:
                        first_innings = play(video,"user",None)
                        show_text_with_timer(video,[f"You Scored {first_innings}","Now Second Innings"])
                        second_innings = play(video,"system",first_innings)
                        if first_innings > second_innings:
                            show_text_with_timer(video,["You Won "])
                            show_end_note(video, [ "Thanks for Playing!"]) 
                        elif first_innings < second_innings:
                            show_text_with_timer(video,["System Won","You Loose  !!!!","Feeling Sad ?"]) 
                            show_end_note(video, [ "Thanks for Playing!"])      
                        else:
                            show_text_with_timer(video,["That's a Tie","What do we do now ?","Play Again"]) 
                            play(video,"user",None)     
                    
                    
        else:
            print("❌ Thumbs up not detected. Exiting...")
            

    else:
        print("❌ Begin screen skipped or failed.")

    video.release()
    cv2.destroyAllWindows()

In [22]:
full()

✅ Result: EVEN
You: 6 System: 2
8
✅ Selected: Batting
🧍 You: 5,  System: 6
🧍 You: 2,  System: 4
🧍 You: 3,  System: 4
🧍 You: 6,  System: 3
🧍 You: 5,  System: 3
🧍 You: 1,  System: 1
✅ Final Total: 21
🧍 You: 5,  System: 3
🧍 You: 1,  System: 5
🧍 You: 6,  System: 4
🧍 You: 5,  System: 5
✅ Final Total: 12
